# Infosys Quarterly Report Analysis

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-vector-stores-faiss llama-index-embeddings-openai

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1845 sha256=7d4fd1b8c2868299705deb5d0956d9fabe779ddae376468730d98bc2c0e0e48b
  Stored in directory: /root/.cache/pip/wheels/a9/16/29/0b45762bf14ad4ba5495cd4ce66c7e326ecb0d5f1edeb7c94d
Successfully built clipboard
  Attempting uninstall: tenacity
    F

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [5]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

In [6]:
import faiss


## Configure LLM service

In [7]:
llm = OpenAI(temperature=0,
             model="chatgpt-4o-latest",
             max_tokens=500)

In [8]:
embed_model = OpenAIEmbedding(embed_batch_size=10)

In [9]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

## Load data
Downloaded from

https://www.infosys.com/investors/reports-filings/quarterly-results.html

In [10]:
q1_2024 = SimpleDirectoryReader(
    input_files=["/content/ifrs-inr-press-release.pdf"]
).load_data()

# Build indices

In [11]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)

In [12]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
q1_2024_index = VectorStoreIndex.from_documents(q1_2024, storage_context=storage_context)

## Build query engines

In [13]:
q1_2024__engine = q1_2024_index.as_query_engine(similarity_top_k=2)

## Run queries

In [14]:
response = q1_2024__engine.query(
    "What is the revenue growth for the quarter?"
)

In [15]:
print(response)

The revenue growth for the quarter was 4.2% year-on-year in constant currency and 1.0% quarter-on-quarter in constant currency.


In [16]:
response = q1_2024__engine.query("Which are some of the key customer wins?")

In [17]:
print(response)

Some of the key customer wins include:

1. **Danske Bank**: Signed a strategic collaboration with Infosys to accelerate its digital transformation initiatives using cloud and AI technologies.
2. **bp**: Signed a Memorandum of Understanding (MoU) with Infosys to make Infosys its primary partner for end-to-end application services.
3. **Aramco**: Partnered with Infosys to enhance HR data analytics, scale automation tools, and improve employee experience through AI technologies.
4. **British Telecom**: Engaged with Infosys Topaz to offer AI-powered predictive analytics for domains like network performance, sustainability, and security.
5. **Walmart Commerce Technologies**: Collaborated with Infosys to deliver scalable omni-channel solutions and streamline the implementation of the Store Assist app.
6. **vidaXL**: Partnered with Infosys to set up scalable and cost-effective business technology support, transitioning 8 agile development teams in under 4 months.
7. **LexisNexis**: Extended i

In [18]:
response = q1_2024__engine.query("What are some of the key achievements in the quarter?")

In [19]:
print(response)

Some of the key achievements in the quarter include:  

- Year-on-year revenue growth of 4.2% in constant currency and sequential growth of 1.0%.  
- Large deal total contract value (TCV) of $2.3 billion, with 56.1% being net new.  
- Operating margin maintained at 20.8%.  
- Basic earnings per share (EPS) increased by 12.4% year-on-year in ₹ terms.  
- Free cash flow (FCF) of ₹5,749 crore, reflecting a 12.6% year-on-year growth and a robust FCF conversion of 96.7% of net profit.  
- Return on equity (ROE) improved by 180 basis points to 32.8%.  
- Attrition rate declined further to 17.3%.  
- Expansion of generative AI capabilities with 80 active client projects under the Topaz AI offering.


In [20]:
response = q1_2024__engine.query("What is the total asset at end of june 2023?")

In [21]:
print(response)

The total assets at the end of June 2023 are ₹131,322 crore.


In [22]:
response = q1_2024__engine.query("What is significant changes in the item in balance sheet in june 2023 from march 2023?")

In [23]:
print(response)

Significant changes in the balance sheet items from March 2023 to June 2023 include:

1. **Earmarked Bank Balance for Dividend**: This increased significantly from `0 crore in March 2023 to `7,262 crore in June 2023.
2. **Current Investments**: Decreased from `6,909 crore in March 2023 to `5,536 crore in June 2023.
3. **Trade Receivables**: Increased from `25,424 crore in March 2023 to `26,183 crore in June 2023.
4. **Unbilled Revenue (Current)**: Decreased from `15,289 crore in March 2023 to `14,628 crore in June 2023.
5. **Other Current Liabilities and Provisions**: Increased significantly from `25,759 crore in March 2023 to `32,863 crore in June 2023.
6. **Lease Liabilities (Non-Current)**: Decreased from `7,057 crore in March 2023 to `6,659 crore in June 2023.
7. **Total Equity**: Decreased slightly from `75,795 crore in March 2023 to `74,828 crore in June 2023.

These changes reflect shifts in the company's financial position over the three-month period.
